# SECURITY AND AUTHENTICITY DURING DATA TRANSMISSION

## BLOCK DIAGRAM

<img src="image2.png">

## OVERVIEW OF OUR PROJECT

<font size="4"><br>STEP 1 - generate global numbers 'g' and 'n'; both are prime numbers

<font size="4"><br>STEP 2 - generate numbers 'a' and 'b'; both are system based, at sender's and receiver's side respectively 

<font size="4"><br>STEP 3 - apply Diffie-Hellman key exchange algorithm

<font size="4"><br>STEP 4 - obtain value of 'alpha' at both end. **'alpha = F(private value)'**

<font size="4"><br>STEP 5 - assign intensity values to the character set belonging to data

<font size="4"><br>STEP 6 - generate a dictionary and an image based on these value at the sender's side. On the other hand, at receiver's side we generate only the dictionary

<font size="4"><br>STEP 7 - obtain value of 'beta' at both end. **'beta = F'(private value)'**

<font size="4"><br>STEP 8 - generate 3 images based on beta value at both ends

<font size="4"><br>STEP 9 - generate a signature by applying DSA at sender's side

<font size="4"><br>STEP 10 - combine all the images (at sender's side)

<font size="4"><br>STEP 11 - send this signature and the encrypted image containing message into network

<font size="4"><br>STEP 12 - take these entities from network, (at receiver) and seperate all the images

<font size="4"><br>STEP 13 - generate the message from the seperated images based on the dictionary generated initially

<font size="4"><br>STEP 14 - check for authenticity

<font size="4"><br>STEP 15 - display appropriate message

## IMPORT LIBRARIES REQUIRED

In [1]:
from Crypto.Util import number
import numpy as np
import random
from PIL import Image
from Crypto.PublicKey import DSA
from Crypto.Signature import DSS
from Crypto.Hash import SHA512

### GENERATING KEYS REQUIRED FOR DSA

In [2]:
def generate():
    key = DSA.generate(2048)
    f = open("public_key.pem", "wb")
    f.write(key.publickey().export_key())
    f.close()
    return key

### SIGNING A MESSAGE

In [3]:
def sign(msg, key):
    message = msg.encode()
    hash_obj = SHA512.new(message)
    signer = DSS.new(key, 'fips-186-3')
    signature = signer.sign(hash_obj)
    return signature

### AUTHENTICATING THE SIGNATURE

In [4]:
def authenticate(msg, signature):
    message = msg.encode()
    f = open("public_key.pem", "r")
    hash_obj = SHA512.new(message)
    pub_key = DSA.import_key(f.read())
    verifier = DSS.new(pub_key, 'fips-186-3')

    try:
        verifier.verify(hash_obj, signature)
        print("The message is authentic.")
    except ValueError:
        print("The message is not authentic.")

### INITIALIZING GLOBAL ENTITIES

In [5]:
global signature_dsa
global img_fuse
global I_g
ar = np.zeros([720, 1280, 3], dtype=np.uint8)
ar.fill(0)
img_fuse = Image.fromarray(ar)
I_g = np.asarray(img_fuse)
global g
g = 0
global n
n = 0
n_length = random.randint(256, 1024)
g = number.getPrime(n_length)
n_length = random.randint(256, 512)
n = number.getPrime(n_length)
global g_power_a_mod_n
g_power_a_mod_n = 0
global g_power_b_mod_n
g_power_b_mod_n = 0
global s

### INITIALIZING SENDER

In [6]:
class SENDER_STEP_1:
    r = random.SystemRandom()
    a = r.randint(1, 255)

    def g_pow_a(self):
        x = (pow(g, SENDER_STEP_1.a) % n)
        return x

### INITIALIZING RECEIVER

In [7]:
class RECEIVER_STEP_1:
    r = random.SystemRandom()
    b = r.randint(1, 255)

    def g_pow_b(self):
        y = (pow(g, RECEIVER_STEP_1.b) % n)
        return y

### TO EXCHANGE THE VALUES g_power_a_mod_n AND g_power_b_mod_n

In [8]:
class1 = SENDER_STEP_1()
class2 = RECEIVER_STEP_1()
g_power_a_mod_n = class1.g_pow_a()
g_power_b_mod_n = class2.g_pow_b()

### SENDER IN ACTION

In [9]:
class SENDER_STEP_2(SENDER_STEP_1):
    def cipher(self):
        g_power_ab = pow(g_power_b_mod_n, SENDER_STEP_2.a) % n
        dict_id = {'A': [],
                   'B': [],
                   'C': [],
                   'D': [],
                   'E': [],
                   'F': [],
                   'G': [],
                   'H': [],
                   'I': [],
                   'J': [],
                   'K': [],
                   'L': [],
                   'M': [],
                   'N': [],
                   'O': [],
                   'P': [],
                   'Q': [],
                   'R': [],
                   'S': [],
                   'T': [],
                   'U': [],
                   'V': [],
                   'W': [],
                   'X': [],
                   'Y': [],
                   'Z': []}
        random.seed(g_power_ab)
        alpha = random.randint(0, 255)
        count_factor = ((alpha) % 11) + 1
        total_assignment_intensities = int(256 / 26)
        remaining_intensities = 256 % 26
        list = []
        a = ((alpha ** 2) * g_power_ab) % 256
        for keys, values in dict_id.items():

            for i in range(0, total_assignment_intensities):
                random.seed(a)
                j = random.randint(0, 255)
                alpha_j = a + j
                while(True):
                    if alpha_j <= 255:
                        if alpha_j not in list:
                            values.append(alpha_j)
                            list.append(alpha_j)
                            break
                        else:
                            alpha_j = alpha_j+1
                            continue
                    else:
                        alpha_j = alpha_j % 256
                a = (j + ((alpha ** 2) * g_power_ab) % 256) % 256
            a = a+count_factor

        for keys, values in dict_id.items():
            print(keys, '   ', values)

        '''list1 = []
        for keys, values in dict_id.items():
            for items in values:
                list1.append(items)
        print(len(list1))
        print(len(set(list1)))'''

        not_in_list = []
        for i in range(0, 256):
            if i not in list:
                not_in_list.append(i)
        # print(not_in_list)
        # print(len(not_in_list))

        s_hash_msg = ''
        for i in not_in_list:
            s_hash_msg = s_hash_msg+str(i)

        random.seed(g_power_ab)
        beta = random.randint(0, 255)
        # print("beta = ", beta)
        # --------------------------

        asci = 0
        s = "NMAMITNIT"
        for i in s:
            asci = asci + ord(i)
        si = str(pow(asci, beta))

        s_hash_msg = s_hash_msg+si
        h = 720
        w = 1280
        array = np.zeros([h, w, 3], dtype=np.uint8)
        pointer = 0
        length = len(s)
        for i in range(h):
            for j in range(w):
                if pointer < length:
                    k = s[pointer]
                    r = random.choice(dict_id[k])
                    g = random.choice(dict_id[k])
                    b = random.choice(dict_id[k])
                    array[i, j] = [r, g, b]
                    pointer = pointer+1
                else:
                    r = random.choice(not_in_list)
                    g = random.choice(not_in_list)
                    b = random.choice(not_in_list)
                    array[i, j] = [r, g, b]
        img_decode = Image.fromarray(array)
        img_decode.show()
        Is1 = np.asarray(img_decode)

        array1 = np.zeros([h, w, 3], dtype=np.uint8)
        beta_1 = (beta * g_power_ab) % 256
        for i in range(h):
            for j in range(w):
                r = (pow((beta_1*alpha), 2) % 256)
                g = (pow((beta_1*alpha+100), 2) % 256)
                b = (pow((beta_1*alpha+150), 2) % 256)
                beta_1 = beta_1 + 50
                array1[i, j] = [r, g, b]
        imgps = Image.fromarray(array1)
        imgps.show()

        Is2 = np.asarray(imgps)

        array2 = np.zeros([h, w, 3], dtype=np.uint8)
        random.seed(beta*g_power_ab*alpha)
        array2.fill(random.randint(0, 255))
        imgps2 = Image.fromarray(array2)
        imgps2.show()
        Is4 = np.asarray(imgps2)

        img_global = Image.open("1.jpg")
        Is3 = np.asarray(img_global)

        Ires = ((Is1+Is2)+(beta*Is3))+Is4
        img_fuse = Image.fromarray(Ires)
        img_fuse.show()

        dsa_key = generate()
        signing = sign(s_hash_msg, dsa_key)

        return Ires, s, signing

### SENDING THE ENCRYPTED MESSAGE AND SIGNATURE INTO NETWORK

In [10]:
class_sender = SENDER_STEP_2()
I_g, s, signature_dsa = class_sender.cipher()
img_fuse = Image.fromarray(I_g)

A     [124, 10, 210, 50, 204, 21, 252, 205, 58]
B     [59, 175, 235, 219, 189, 57, 178, 143, 222]
C     [206, 91, 191, 88, 216, 207, 60, 176, 236]
D     [165, 24, 169, 147, 139, 156, 242, 22, 26]
E     [220, 173, 217, 170, 148, 140, 157, 243, 23]
F     [97, 36, 237, 9, 149, 141, 158, 244, 25]
G     [98, 37, 238, 11, 150, 142, 159, 245, 27]
H     [99, 38, 239, 12, 151, 144, 160, 246, 28]
I     [100, 39, 240, 13, 152, 145, 161, 247, 29]
J     [101, 40, 241, 14, 153, 146, 162, 248, 30]
K     [102, 41, 249, 15, 154, 155, 163, 250, 31]
L     [103, 42, 251, 16, 164, 166, 167, 253, 32]
M     [104, 43, 254, 17, 168, 171, 172, 255, 33]
N     [105, 44, 0, 18, 174, 177, 179, 1, 34]
O     [106, 45, 2, 19, 180, 181, 182, 3, 35]
P     [107, 46, 4, 20, 183, 184, 185, 5, 47]
Q     [108, 48, 6, 49, 186, 187, 188, 7, 51]
R     [109, 52, 8, 53, 190, 192, 193, 54, 55]
S     [110, 56, 61, 62, 194, 195, 196, 63, 64]
T     [111, 65, 66, 67, 197, 198, 199, 68, 69]
U     [112, 70, 71, 72, 200, 201, 202, 73, 74

### RECEIVER IN ACTION

In [11]:
class RECEIVER_STEP_2(RECEIVER_STEP_1):
    def decipher(self):
        g_power_ab = pow(g_power_a_mod_n, RECEIVER_STEP_2.b) % n
        dict_id = {'A': [],
                   'B': [],
                   'C': [],
                   'D': [],
                   'E': [],
                   'F': [],
                   'G': [],
                   'H': [],
                   'I': [],
                   'J': [],
                   'K': [],
                   'L': [],
                   'M': [],
                   'N': [],
                   'O': [],
                   'P': [],
                   'Q': [],
                   'R': [],
                   'S': [],
                   'T': [],
                   'U': [],
                   'V': [],
                   'W': [],
                   'X': [],
                   'Y': [],
                   'Z': []}
        random.seed(g_power_ab)
        alpha = random.randint(0, 255)
        count_factor = ((alpha) % 11) + 1
        total_assignment_intensities = int(256 / 26)
        remaining_intensities = 256 % 26
        list = []
        a = ((alpha ** 2) * g_power_ab) % 256

        for keys, values in dict_id.items():

            for i in range(0, total_assignment_intensities):
                random.seed(a)
                j = random.randint(0, 255)
                alpha_j = a + j
                while(True):
                    if alpha_j <= 255:
                        if alpha_j not in list:
                            values.append(alpha_j)
                            list.append(alpha_j)
                            break
                        else:
                            alpha_j = alpha_j+1
                            continue
                    else:
                        alpha_j = alpha_j % 256
                a = (j + ((alpha ** 2) * g_power_ab) % 256) % 256
            a = a+count_factor

        for keys, values in dict_id.items():
            print(keys, '   ', values)

        '''list1 = []
        for keys, values in dict_id.items():
            for items in values:
                list1.append(items)
        print(len(list1))
        print(len(set(list1)))'''

        not_in_list = []
        for i in range(0, 256):
            if i not in list:
                not_in_list.append(i)
        # print(not_in_list)
        # print(len(not_in_list))

        s_hash_msg = ''
        for i in not_in_list:
            s_hash_msg = s_hash_msg+str(i)

        random.seed(g_power_ab)
        beta = random.randint(0, 255)
        # print("beta = ", beta)

        # ------------------------

        h = 720
        w = 1280
        array1 = np.zeros([h, w, 3], dtype=np.uint8)
        beta_1 = (beta * g_power_ab) % 256

        for i in range(h):
            for j in range(w):
                r = (pow((beta_1*alpha), 2) % 256)
                g = (pow((beta_1*alpha+100), 2) % 256)
                b = (pow((beta_1*alpha+150), 2) % 256)
                beta_1 = beta_1 + 50
                array1[i, j] = [r, g, b]
        imgps = Image.fromarray(array1)
        imgps.show()
        Ir2 = np.asarray(imgps)

        array3 = np.zeros([h, w, 3], dtype=np.uint8)
        random.seed(beta*g_power_ab*alpha)
        array3.fill(random.randint(0, 255))
        imgps3 = Image.fromarray(array3)
        imgps3.show()
        Ir4 = np.asarray(imgps3)

        img_global = Image.open("1.jpg")
        Ir3 = np.asarray(img_global)

        Ires = np.asarray(img_fuse)

        Idecode = (((Ires - Ir4) - (beta * Ir3)) - Ir2)
        img_res = Image.fromarray(Idecode)
        img_res.show()
        message = ''
        pix = img_res.load()

        for j in range(h):
            for i in range(w):
                for keys, values in dict_id.items():
                    if set(pix[i, j]).issubset(values):
                        message = message+keys

        asci = 0
        for i in message:
            asci = asci + ord(i)
        si = str(pow(asci, beta))

        s_hash_msg = s_hash_msg+si
        authenticate(s_hash_msg, signature_dsa)
        return(message)

### GENERATING RESULT

In [12]:
class_receiver = RECEIVER_STEP_2()
m = class_receiver.decipher()

A     [97, 196, 9, 1, 219, 249, 156, 206, 10]
B     [27, 225, 227, 223, 62, 146, 149, 99, 207]
C     [222, 157, 187, 128, 208, 11, 188, 129, 209]
D     [143, 107, 2, 220, 250, 158, 210, 12, 189]
E     [13, 190, 228, 224, 63, 147, 150, 100, 211]
F     [226, 159, 191, 130, 212, 14, 192, 131, 213]
G     [144, 108, 3, 221, 251, 160, 214, 15, 193]
H     [16, 194, 229, 230, 64, 148, 151, 101, 215]
I     [231, 161, 195, 132, 216, 17, 197, 133, 217]
J     [145, 109, 4, 232, 252, 162, 218, 18, 198]
K     [19, 199, 233, 234, 65, 152, 153, 102, 235]
L     [236, 163, 200, 134, 237, 20, 201, 135, 238]
M     [154, 110, 5, 239, 253, 164, 240, 21, 202]
N     [22, 203, 241, 242, 66, 155, 165, 103, 243]
O     [244, 166, 204, 136, 245, 23, 205, 137, 246]
P     [167, 111, 6, 247, 254, 168, 248, 24, 255]
Q     [25, 0, 7, 8, 67, 169, 170, 104, 26]
R     [28, 171, 29, 138, 30, 31, 32, 139, 33]
S     [172, 112, 34, 35, 36, 173, 37, 38, 39]
T     [40, 41, 42, 43, 68, 174, 175, 105, 44]
U     [45, 176, 46, 140,

In [13]:
print(m)

NMAMITNIT


### FINAL OUTPUT